# Benchmark for Clustering

In [1]:
"""Load/import helper functions"""

import time
import random
from LocalPopular import locally_popular_clustering, extract_labels_from_communities, time_tester, calculate_scores_clustering, \
    locally_popular_clustering_with_euclid_graphs

from GraphFunctions import generate_agents, calculate_euclidian_relationships, create_graph, \
    my_make_circles, create_graphs_euclid, create_graphs_kNN, \
    generate_graph,create_graphs_hop_distance, create_graphs_hop_distance_abs,randomize_graph_pos_labels

from PlotHelperFunctions import plot_clustering, plot_stuff

from sklearn.cluster import KMeans, DBSCAN
from sklearn.datasets import make_moons, load_breast_cancer, load_iris
from sklearn.metrics import rand_score
import numpy as np
import networkx as nx
from scipy.spatial import distance

from community_detection.leiden import leiden
from community_detection.louvain import louvain
from community_detection.quality_functions import CPM, Modularity



## Create Graphs

In [2]:
repetitions = 1    #Number of random isomorph permutation for each graph


moon_agents,moon_truth = make_moons(n_samples=300, noise=0.05)
moon_perm_agents = []
moon_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_pos_labels(moon_agents,moon_truth)
    moon_perm_agents += [g]
    moon_perm_truth += [t]



circle_agents, circle_truth = my_make_circles(300)

circle_perm_agents = []
circle_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_pos_labels(circle_agents,circle_truth)
    circle_perm_agents += [g]
    circle_perm_truth += [t]

cancer = load_breast_cancer()

cancer_agents = cancer['data']
cancer_truth = cancer['target']

cancer_perm_agents = []
cancer_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_pos_labels(cancer_agents,cancer_truth)
    cancer_perm_agents += [g]
    cancer_perm_truth += [t]

iris = load_iris()
iris_agents = iris['data']
iris_truth = iris['target']


iris_perm_agents = []
iris_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_pos_labels(iris_agents,iris_truth)
    iris_perm_agents += [g]
    iris_perm_truth += [t]


## Run the algorithms


In [3]:
import itertools
import timeit

f = 0.2   #f-bound
e = 0.3   #e-bound

data = [ moon_perm_agents,circle_perm_agents,cancer_perm_agents,iris_perm_agents]
expected_clusters = [2,3,2,3]
graph_names = ['Moons','My 3 Circles', 'Cancer', 'Iris']
graph_truths =  [moon_perm_truth,circle_perm_truth,cancer_perm_truth,iris_perm_truth]


kmeans = lambda agents, clusters: KMeans(n_clusters = clusters).fit_predict(agents)
dbscan = lambda agents, clusters: DBSCAN(eps=0.2, min_samples=5).fit_predict(agents)

kmeans_out = None
dbscan_out = None

lp_a_b =lambda agents, initial_clustering, pre: locally_popular_clustering_with_euclid_graphs(agents, f, e, initial_clustering,mode='B',pre=pre)
lp_a_f =lambda agents, initial_clustering, pre: locally_popular_clustering_with_euclid_graphs(agents, f, e, initial_clustering,mode='F',pre=pre)
lp_a_e =lambda agents, initial_clustering, pre: locally_popular_clustering_with_euclid_graphs(agents, f, e, initial_clustering,mode='E',pre=pre)

algorithms = [ kmeans, dbscan,lp_a_b,lp_a_f,lp_a_e]
algo_names = [ 'kmeans', 'dbscan','LP (Balanced) Heuristic',\
               'LP (Friend-Oriented) Heuristic','LP (Enemy-Averse) Heuristic']
is_lp_heuristic = [False,False,True, True, True]

collected_data = {}

for ((graph, g_name,clusters,truth), (algo, a_name,lp_heuristic)) in \
    itertools.product(zip(data, graph_names, expected_clusters,graph_truths), zip(algorithms, algo_names,is_lp_heuristic)):

        
    agents = graph

    if lp_heuristic:
        # start with everyone alone
        a_name_modified = a_name + ' starting with everyone alone'
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        test_callable = lambda a: list(algo(a,len(agents[0]),None).values())
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_clustering(outputs,truth,agents)
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

        # start with random clustering 

        a_name_modified = a_name + ' starting with predicted number of clusters'
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        test_callable = lambda a: list(algo(a,clusters,None).values())
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_clustering(outputs,truth,agents)
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))


        # start with the output of k-means
        a_name_modified = a_name + ' starting with the output of k-means'
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        test_callable = lambda a: list(algo(a,clusters,kmeans).values())
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_clustering(outputs,truth,agents)

        rand_score_with_init = sum(rand_score(out, k) for out, k in zip(outputs, kmeans_out)) / len(outputs)
        scores['Rand Score with initial clustering'] = rand_score_with_init
        
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

        # start with the output of dbscan
        a_name_modified = a_name + ' starting with the output of dbscan'
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        test_callable = lambda a: list(algo(a,clusters,kmeans).values())
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)

        rand_score_with_init = sum(rand_score(out, db) for out, db in zip(outputs, dbscan_out)) / len(outputs)
        scores['Rand Score with initial clustering'] = rand_score_with_init
        
        scores = calculate_scores_clustering(outputs,truth,agents)
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

            
    else:

        print(f"Running {a_name} on {g_name} … ", end='')
        test_callable = lambda a : algo(a, clusters)
            
        times,outputs = time_tester(test_callable,graph)
        if(algo == kmeans):
            kmeans_out = outputs
        if(algo == dbscan):
            dbscan_out = outputs
        avg_time = sum(times)/len(times)
        scores = calculate_scores_clustering(outputs,truth,agents)
        scores['Time'] = avg_time
        
        collected_data[(a_name,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))



Running kmeans on Moons … execution time: ~ 0.103301 seconds.
Rand Index : ~ 0.6171460423634336
Silhouette Score : ~ 0.486834021194243
Davies Bouldin Score : ~ 0.7836909776607694
Time : ~ 0.10330139985308051
Running dbscan on Moons … execution time: ~ 0.002337 seconds.
Rand Index : ~ 1.0
Silhouette Score : ~ 0.3302053021892839
Davies Bouldin Score : ~ 1.1639050416879664
Time : ~ 0.002337199868634343
Running LP (Balanced) Heuristic starting with everyone alone on Moons … execution time: ~ 3.911046 seconds.
Rand Index : ~ 0.6416945373467112
Silhouette Score : ~ 0.1798390466309817
Davies Bouldin Score : ~ 0.70586428941262
Time : ~ 3.9110460998490453
Running LP (Balanced) Heuristic starting with predicted number of clusters on Moons … execution time: ~ 0.355177 seconds.
Rand Index : ~ 0.7730434782608696
Silhouette Score : ~ 0.4546201673167128
Davies Bouldin Score : ~ 0.8500009448004037
Time : ~ 0.3551773000508547
Running LP (Balanced) Heuristic starting with the output of k-means on Moons 

## Gather the numbers

We can use the collected_data dictionairy to build a table for better comparison


In [4]:
print(collected_data)

{('kmeans', 'Moons'): {'Rand Index': np.float64(0.6171460423634336), 'Silhouette Score': np.float64(0.486834021194243), 'Davies Bouldin Score': np.float64(0.7836909776607694), 'Time': 0.10330139985308051}, ('dbscan', 'Moons'): {'Rand Index': 1.0, 'Silhouette Score': np.float64(0.3302053021892839), 'Davies Bouldin Score': np.float64(1.1639050416879664), 'Time': 0.002337199868634343}, ('LP (Balanced) Heuristic starting with everyone alone', 'Moons'): {'Rand Index': np.float64(0.6416945373467112), 'Silhouette Score': np.float64(0.1798390466309817), 'Davies Bouldin Score': np.float64(0.70586428941262), 'Time': 3.9110460998490453}, ('LP (Balanced) Heuristic starting with predicted number of clusters', 'Moons'): {'Rand Index': np.float64(0.7730434782608696), 'Silhouette Score': np.float64(0.4546201673167128), 'Davies Bouldin Score': np.float64(0.8500009448004037), 'Time': 0.3551773000508547}, ('LP (Balanced) Heuristic starting with the output of k-means', 'Moons'): {'Rand Index': np.float64(